In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from datetime import datetime, time
import pandas_datareader.data as pdr
import yfinance as yf 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [6]:
def stock_data(ticker, start):
    try:
        prices = yf.download(ticker, start, auto_adjust=True)['Close']
    except Exception as e:
        print(f"Error: {e}")
    return prices
    

In [ ]:
def dual_momemtum(prices, stocks, market, mom_lookback=126, ma_window = 200, top_n=5):
    """Calculating 6 month dual momentum with monthly rebalancing"""


     #Monthly rebalancing to trade monthly
    prices_rb = prices.resample("M").last()

    #Absolute Momemtum for the market
    abs_mom = prices_rb[market].pct_change(mom_lookback)
    trend = (prices_rb[market] > prices_rb[market].rolling(ma_window).mean())
    market_signal = (abs_mom > 0) & trend

    #Relative momentum for stocks
    stocks_return = prices_rb[stocks].pct_change(mom_lookback)

    #Ranking stocks
    ranks = stocks_return.rank(axis = 1, ascending= False)
    top_stocks = ranks <= top_n

    #monthly weights
   
    weights_rb = pd.DataFrame(0.0, index= prices_rb.index, columns=prices.columns)
    "this step is initializing weight matrix. rows are rebalance dates . Cash is represented as all zeros"

    for date in prices_rb.index:
        if market_signal.loc[date]:
            selected = top_stocks.loc[date]
            if selected.sum() > 0:
                weights_rb.loc[date, stocks] = (selected / selected.sum()) #assigning equal weight to selected stocks

    #Daily tradable weights
    weights = ( weights_rb.reindex(prices.index).ffill().shift(1).fillna(0))

    """reindex -> expands weights from monthly to daily and add Nads to non-rebalace days.
    ffill() - forward-fills last rebelance weights and portfolio is held constant between rebalance dates
    shift(1) - signals use today's close price (prevent lookahead bias)
    fillna(0) - before first signal. everything is in cash which is presented by 0"""

    #Strategy returns
    returns = prices[stocks].pct_change()
    strategy_returns =( weights * returns).sum(axis = 1)
    equity_curve = (1+ strategy_returns).cumprod()



    #plot equity curves
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=equity_curve.index, y = equity_curve, mode="lines", name='Dual Momentum'))

    fig.update_layout(title = "Dual Momentum Equit curve",
                      xaxis_title="Date", yaxis_title ='Portfolio value', yaxis_type = "log",
                      template="plotly_white", hovermoce ='x unified')
    fig.show()

    #Plot drawdown
    rolling_max = equity_curve.cummax()
    drawdown = equity_curve / rolling_max -1
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x = drawdown.index, y = drawdown, fill="tozeroy", name = "Drawdown"))
    fig1.update_layout(title = "Drawdown",
                       xaxis_title ='Date', yaxis_title = "Drawdown", template = "plotly_white",
                       hovermode="x unified")
    
    fig1.show()

    #Market Regime plot
    prices_market = prices[market]
    fig2 = go.Figure()

    fig2.add_trace(
    go.Scatter(
        x=prices_market.index,
        y=prices_market,
        mode="lines",
        name="Market "
        ))
                        

    fig2.add_trace(
    go.Scatter(
        x=market_signal[market_signal].index,
        y=prices_market.loc[market_signal],
        mode="markers",
        name="Risk ON",
        marker=dict(size=6)
            )
                    )

    fig2.update_layout(
    title="Market Regime Filter",
    xaxis_title="Date",
    yaxis_title="Market Price",
    template="plotly_white"
                )

    fig2.show()


    return equity_curve